In [ ]:
#Import necessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.models import resnet50
from torchsummary import summary

#Set up the notebook
%matplotlib inline
plt.style.use('default')

In [ ]:
import cv2
import numpy as np
import